In [1]:
from lilac.jobs.job_base import BasicSeedJob

/Users/hikaru/github/ds-monorepo/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# 基本的にはyamlを読み込めばいいけど、
# pathをipynbからのパスに変更、model_dirとpred_onlyを変えるあたりが変更ポイント
params={
    "train_path": "data/train_v5.csv",
    "test_path": "data/test_v5.csv",
    "features_dir": "data/features/v5",
    "target_col": "price_log",
    "base_col": "area_log",
    "n_estimators": 100,
    "fold_num": 2,
    "seed": 42,
    "depth": 8,
    "min_child_samples": 200,
    "folds_gen_str": "group",
    "group_key_col": "ordered",
    "model_str": "lgbm_diff_mae",
    "evaluator_str": "mae",
    "unused_cols": ["id", 
    "ordered",
    "built_year", # 西暦を使っているから
    "area", # area_logを使っているから 
    "nearest_min", # nearest_min_logを使っているから
    "age" # ageを使っているから
    ],
    "target_enc_cols": ["nearest_sta","district"],
    "log_target_on_target_enc": False,
    "features_settings":[
        {
            "model_str":"group",
            "params": {
                "group_keys": ["city","district","nearest_sta"],
                "agg_func_list":["mean"],
                "input_cols": ["built_year_seireki","area","nearest_min"]
            }
        }
    ],
    "model_dir":"data/models/sample_nb",
    # ここをTrueにすると予測だけ行う
    "pred_only":True,
}

In [9]:
output=BasicSeedJob(**params).run()

Extracting required params in FeatureGeneratorsFactory.
[WARNING] parameter 'do_add_diff' is not specified. So default is used.
[WARNING] parameter 'do_add_ratio' is not specified. So default is used.
Extracting required params in FoldsGeneratorFactory.
Extracting required params in TrainerFactory.
Extracting required params in EvaluatorFactory.
[1/1] GroupFeatures
Loading _GroupFeatures_430f5abfeca359abe0b7edd5a9ffa278 (train)...
Loading _GroupFeatures_430f5abfeca359abe0b7edd5a9ffa278 (test)...
Loading _GroupFeatures_d81ac306b4b857898d07605673306bd8 (train)...
Loading _GroupFeatures_d81ac306b4b857898d07605673306bd8 (test)...
Loading _GroupFeatures_451639cd3fe572fa2ec47f99d5ee5e03 (train)...
Loading _GroupFeatures_451639cd3fe572fa2ec47f99d5ee5e03 (test)...
Loading _GroupFeatures_486a343bdae1287fefe39d7a562144ec (train)...
Loading _GroupFeatures_486a343bdae1287fefe39d7a562144ec (test)...
Loading _GroupFeatures_c6a68002c5d26ce7edf81fb999e58546 (train)...
Loading _GroupFeatures_c6a68002c5

In [10]:
output.keys()

dict_keys(['raw_pred', 'pred'])

In [11]:
output

{'raw_pred': [6.774537938555854,
  7.269824990439315,
  7.620243330089769,
  6.84864194771812,
  6.5789363439974515,
  7.655797353142642,
  7.583552012746801,
  7.4225337701100855,
  7.323952216170328,
  7.26924285603035,
  7.0992811388040185,
  7.348354793469641,
  7.453195748677067,
  7.381483286386375,
  7.183149150204729,
  7.197548299535686,
  7.348392742809364,
  6.899864175768779,
  7.405854764422432,
  6.900678597374756,
  7.433591709935314,
  7.369466802719927,
  7.442265791996476,
  7.358466927754135,
  7.543300427357048,
  7.286515522840362,
  7.4301562383927395,
  7.248352029355834,
  7.0763805023220705,
  6.883248605109989,
  7.3408037805779,
  7.420433039444365,
  7.44474438041485,
  7.489833906446921,
  7.644358318537522,
  7.464528128791697,
  7.6300211699804725,
  7.434727468491266,
  7.511997836350719,
  6.92242858413628,
  7.42667102347405,
  7.483035616089689,
  7.430449336108653,
  7.363282004592802,
  7.467245482041026,
  7.045974571468166,
  7.3429069489443055,
 